In [1]:
import os
import openai

In [2]:
os.environ["OPENAI_API_KEY"] = "sk-4NfQ6e8e4BwaNF8bE64150E16c3e4dC897278eEb8209E5C8"
openai.api_key = os.environ["OPENAI_API_KEY"]

openai.api_base = "https://api.xhrai.com/v1"
api_key="sk-4NfQ6e8e4BwaNF8bE64150E16c3e4dC897278eEb8209E5C8"
api_base = "https://api.xhrai.com/v1"

In [3]:
from IPython.display import Markdown, display

In [4]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    select,
)

In [5]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

In [6]:
# create city SQL table
table_name = "bop_test_cases"
bop_test_cases = Table(
    table_name,
    metadata_obj,
    Column("id", Integer, primary_key=True, autoincrement=True),
    Column("branch", String(16), nullable=False),
    Column("payment_type", String(16), nullable=True),
    Column("customer_transaction_code1", String(16), nullable=True),
    Column("customer_transaction_amount1", Float, nullable=True),
    Column("customer_transaction_postscript1", String(16), nullable=True),
    Column("final_transaction_code1", String(16), nullable=True),
    Column("final_transaction_amount1", Float, nullable=True),
    Column("final_transaction_postscript1", String(16), nullable=True),
)
metadata_obj.create_all(engine)

In [7]:
from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI

In [8]:
llm = OpenAI(temperature=0.1, model="gpt-4-turbo",api_base = api_base,api_key = api_key)

In [9]:
sql_database = SQLDatabase(engine, include_tables=["bop_test_cases"])

In [10]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String

excel_file = "./bop.xlsx"  
df = pd.read_excel(excel_file)

In [11]:
sql_database = SQLDatabase(engine, include_tables=["bop_test_cases"])
from sqlalchemy import insert

# rows = [
#     {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
#     {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
#     {
#         "city_name": "Chicago",
#         "population": 2679000,
#         "country": "United States",
#     },
#     {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
# ]
# for row in rows:
#     stmt = insert(city_stats_table).values(**row)
#     with engine.begin() as connection:
#         cursor = connection.execute(stmt)
with engine.begin() as connection:
    for index, row in df.iterrows():
        stmt = insert(bop_test_cases).values(
            branch=row["Branch"],
            payment_type=row["付款类型"],
            customer_transaction_code1=row["客户提供的交易编码1"],
            customer_transaction_amount1=row["客户提供的交易金额1"],
            customer_transaction_postscript1=row["客户提供的交易附言1"],
            final_transaction_code1=row["最终的交易编码1"],
            final_transaction_amount1=row["最终的相应金额1"],
            final_transaction_postscript1=row["最终的交易附言1"],
        )
        connection.execute(stmt)

In [12]:
# view current table
stmt = select(
    bop_test_cases.c.branch,
    bop_test_cases.c.payment_type,
    bop_test_cases.c.customer_transaction_code1,
    bop_test_cases.c.customer_transaction_amount1,
    bop_test_cases.c.customer_transaction_postscript1,
    bop_test_cases.c.final_transaction_code1,
    bop_test_cases.c.final_transaction_amount1,
    bop_test_cases.c.final_transaction_postscript1,
).select_from(bop_test_cases)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('SHH', '其他', '222011', 25.0, '海出', '222011-涉及我国出口的海洋货运服务', 25.0, '出口海运费'), ('XIA', '货到付款', '121010', 62.6, '一般贸易电子器件货款', '121010-一般贸易', 63.0, '一般贸易电子器件的货款'), ('BJG', '其他', '225030', 135.09, '再保险支出财产险', '225030', 135.0, '再保险支出财产险'), ('SHH', '其他', '228039', 163.5, '店铺门禁技术费', '228039-其他技术服务', 164.0, '店铺门禁技术费'), ('SHH', '货到付款', '121010', 366.97, '弹簧', '121010-一般贸易', 367.0, '一般贸易弹簧'), ('XIA', '货到付款', '121020', 696.76, '进料加工电子器件', '121020-进料加工贸易', 697.0, '进料加工电子器件的货款'), ('SHH', '其他', '228990', 746.73, '巧克力包装服务费', '228990-上述未提及的其他商业服务', 747.0, '巧克力包装服务费'), ('GZH', '其他', '222013', 810.0, '不涉及我国进出口货物的海运费支出', '222013-不涉及我国进出口的海洋货运服务', 810.0, '不涉及我国进出口货物的海运费支出'), ('SHH', '其他', '228050', 915.07, '干式拉丝润滑粉产品销售佣金', '228050-货物或服务交易佣金及相关服务', 915.0, '干式拉丝润滑粉产品销售佣金'), ('SHH', '其他', '222011', 1050.0, '出口海运费', '222011-涉及我国出口的海洋货运服务', 1050.0, '出口海运费'), ('GZH', '货到付款', '121010', 1170.0, '一般贸易(真空泵机械密封)', '121010-一般贸易', 1170.0, '一般贸易项下进口真空泵机械密封货款支出'), ('SHH', '其他', '321000', 1850.0, '外籍员工薪资福利', '321000-职工报酬（

In [13]:
from sqlalchemy import text

with engine.connect() as con:
    rows = con.execute(text("SELECT customer_transaction_amount1 from bop_test_cases"))
    for row in rows:
        print(row)

(25.0,)
(62.6,)
(135.09,)
(163.5,)
(366.97,)
(696.76,)
(746.73,)
(810.0,)
(915.07,)
(1050.0,)
(1170.0,)
(1850.0,)
(2000.0,)
(2065.0,)
(2206.5,)
(2310.0,)
(2402.27,)
(2856.6,)
(2900.0,)
(3136.34,)
(3220.83,)
(3531.0,)
(3700.01,)
(3795.0,)
(3800.0,)
(3813.0,)
(3833.5,)
(4065.0,)
(4161.71,)
(4493.9,)
(4753.11,)
(5429.43,)
(5500.0,)
(5639.09,)
(5834.53,)
(6434.06,)
(6436.6,)
(7301.25,)
(7776.0,)
(7787.94,)
(8721.39,)
(9000.0,)
(9074.79,)
(10150.66,)
(12000.0,)
(14364.0,)
(16221.72,)
(16762.76,)
(16845.36,)
(17022.78,)
(19570.0,)
(21294.9,)
(21585.0,)
(21685.0,)
(21863.31,)
(24792.64,)
(28662.0,)
(28813.5,)
(30717.58,)
(33744.0,)
(36605.15,)
(38860.8,)
(39283.2,)
(41815.28,)
(43025.0,)
(49133.31,)
(53120.0,)
(53360.05,)
(53550.0,)
(64750.0,)
(67000.0,)
(68411.84,)
(70463.25,)
(77547.16,)
(97081.22,)
(99000.0,)
(111054.02,)
(171000.0,)
(216100.0,)
(258324.5,)
(258630.0,)
(264150.0,)
(300262.69,)
(316602.0,)
(391058.31,)
(490196.62,)
(537282.42,)
(562336.49,)
(609801.26,)
(802831.0,)
(811008.

In [14]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["bop_test_cases"], llm=llm
)
query_str = "返回'客户提供的交易金额1'第2小的branch以及'客户提供的交易金额1'"
# query_str = "Return 'customer provided by the transaction note 1' with branch=XIA and 'customer provided by the transaction amount 1'=62.6 ."
# query_str = "Returns the difference between the sum of all final_transaction_amount1 for data with payment_type as '预付货款' and sum of final_transaction_amount1 for those with branch 'TJN'."
response = query_engine.query(query_str)


1111111111
[ChatMessage(role=<MessageRole.USER: 'user'>, content="Given an input question, first create a syntactically correct sqlite query , then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a few relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:\n\nQuestion: Question here\nSQLQuery: SQL Query \nSQLResult: Result of the SQLQuery\nAnswer: Final answer here\n\nOnly use tables listed below.\nTable 'bop_test_cases' has columns: id (INTEGER), branch (VARCHAR(16)), payment_type (VARCHAR(16)), customer_tr

In [15]:
display(Markdown(f"{response}"))

The second smallest 'customer_transaction_amount1' is 62.6, and it is associated with the branch 'XIA'.

In [18]:
import json
from llama_index.core.retrievers import NLSQLRetriever
import time
from llama_index.core.response.notebook_utils import display_source_node

with open('output.json', 'r' ,encoding='GBK') as f:
    data = json.load(f)
questions = [data[i]['question'] for i in range(len(data))]
sql_queries = [data[i]['sql_query'] for i in range(len(data))]
start_time = time.time()
outputs = []
with open('results.txt', 'w', encoding='utf-8') as file:
    for question in questions:
        nl_sql_retriever = NLSQLRetriever(
            sql_database, tables=["bop_test_cases"], return_raw=True,llm=llm
        )
        results = nl_sql_retriever.retrieve(question)
        file.write(f"Question: {question}\n")
        for n in results:
            outputs.append(display_source_node(n, source_length=10000))
        file.write(f"Answer: {outputs[-1]}\n")
end_time = time.time()
execution_time = end_time - start_time
print("程序运行时间为：", execution_time, "秒")

1111111111
[ChatMessage(role=<MessageRole.USER: 'user'>, content="Given an input question, first create a syntactically correct sqlite query , then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a few relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:\n\nQuestion: Question here\nSQLQuery: SQL Query \nSQLResult: Result of the SQLQuery\nAnswer: Final answer here\n\nOnly use tables listed below.\nTable 'bop_test_cases' has columns: id (INTEGER), branch (VARCHAR(16)), payment_type (VARCHAR(16)), customer_tr

**Node ID:** 85dc8640-5af0-45ba-9bc2-ddabd920ddf4<br>**Similarity:** None<br>**Text:** [('SHH', 422104400.0), ('BJG', 90735372.47), ('BJG', 29300000.0), ('BJG', 11062394.84), ('BJG', 3400000.0), ('SHH', 1800546.42), ('SHH', 1400000.0), ('BJG', 811008.0), ('GZH', 802831.0), ('BJG', 609801.26), ('BJG', 562336.49), ('GZH', 537282.42), ('GZH', 490196.62), ('SHH', 391058.31), ('BJG', 316602.0), ('GZH', 300262.69), ('SHH', 264150.0), ('GZH', 258630.0), ('SHH', 258324.5), ('CQG', 216100.0), ('SHH', 171000.0), ('SHH', 111054.02), ('GZH', 99000.0), ('SHH', 97081.22), ('SHH', 77547.16), ('GZH', 70463.25), ('XIA', 68411.84), ('CQG', 67000.0), ('SHH', 64750.0), ('GZH', 53550.0), ('SHH', 53360.05), ('CQG', 53120.0), ('SHH', 49133.31), ('GZH', 43025.0), ('GZH', 41815.28), ('XIA', 39283.2), ('SHH', 38860.8), ('GZH', 36605.15), ('SHH', 33744.0), ('GZH', 30717.58), ('XIA', 28813.5), ('SHH', 28662.0), ('GZH', 24792.64), ('SHH', 21863.31), ('SHH', 21685.0), ('CQG', 21585.0), ('SHH', 21294.9), ('TJN', 19570.0), ('SHH', 17022.78), ('BJG', 16845.36), ('SHH', 16762.76), ('GZH', 16221.72), ('SHH', 14364.0), ('GZH', 12000.0), ('BJG', 10150.66), ('BJG', 9074.79), ('BJG', 9000.0), ('SHH', 8721.39), ('BJG', 7787.94), ('CQG', 7776.0), ('SHH', 7301.25), ('SHH', 6436.6), ('SHH', 6434.06), ('SHH', 5834.53), ('GZH', 5639.09), ('SHH', 5500.0), ('BJG', 5429.43), ('SHH', 4753.11), ('SHH', 4493.9), ('BJG', 4161.71), ('SHH', 4065.0), ('GZH', 3833.5), ('XIA', 3813.0), ('SHH', 3800.0), ('SHH', 3795.0), ('SHH', 3700.01), ('GZH', 3531.0), ('SHH', 3220.83)]<br>

1111111111
[ChatMessage(role=<MessageRole.USER: 'user'>, content="Given an input question, first create a syntactically correct sqlite query , then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a few relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:\n\nQuestion: Question here\nSQLQuery: SQL Query \nSQLResult: Result of the SQLQuery\nAnswer: Final answer here\n\nOnly use tables listed below.\nTable 'bop_test_cases' has columns: id (INTEGER), branch (VARCHAR(16)), payment_type (VARCHAR(16)), customer_tr

**Node ID:** 84b0a845-cd2c-432c-9dae-aead58e37a5d<br>**Similarity:** None<br>**Text:** [('SHH', 422104400.0), ('BJG', 90735372.47), ('BJG', 29300000.0), ('BJG', 11062394.84), ('BJG', 3400000.0), ('SHH', 1800546.42), ('SHH', 1400000.0), ('BJG', 811008.0), ('GZH', 802831.0), ('BJG', 609801.26), ('BJG', 562336.49), ('GZH', 537282.42), ('GZH', 490196.62), ('SHH', 391058.31), ('BJG', 316602.0), ('GZH', 300262.69), ('SHH', 264150.0), ('GZH', 258630.0), ('SHH', 258324.5), ('CQG', 216100.0), ('SHH', 171000.0), ('SHH', 111054.02), ('GZH', 99000.0), ('SHH', 97081.22), ('SHH', 77547.16), ('GZH', 70463.25), ('XIA', 68411.84), ('CQG', 67000.0), ('SHH', 64750.0), ('GZH', 53550.0), ('SHH', 53360.05), ('CQG', 53120.0), ('SHH', 49133.31), ('GZH', 43025.0), ('GZH', 41815.28), ('XIA', 39283.2), ('SHH', 38860.8), ('GZH', 36605.15), ('SHH', 33744.0), ('GZH', 30717.58), ('XIA', 28813.5), ('SHH', 28662.0), ('GZH', 24792.64), ('SHH', 21863.31), ('SHH', 21685.0), ('CQG', 21585.0), ('SHH', 21294.9), ('TJN', 19570.0), ('SHH', 17022.78), ('BJG', 16845.36), ('SHH', 16762.76), ('GZH', 16221.72), ('SHH', 14364.0), ('GZH', 12000.0), ('BJG', 10150.66), ('BJG', 9074.79), ('BJG', 9000.0), ('SHH', 8721.39), ('BJG', 7787.94), ('CQG', 7776.0), ('SHH', 7301.25), ('SHH', 6436.6), ('SHH', 6434.06), ('SHH', 5834.53), ('GZH', 5639.09), ('SHH', 5500.0), ('BJG', 5429.43), ('SHH', 4753.11), ('SHH', 4493.9), ('BJG', 4161.71), ('SHH', 4065.0), ('GZH', 3833.5), ('XIA', 3813.0), ('SHH', 3800.0), ('SHH', 3795.0), ('SHH', 3700.01), ('GZH', 3531.0), ('SHH', 3220.83), ('BJG', 3136.34), ('SHH', 2900.0), ('XIA', 2856.6), ('SHH', 2402.27), ('SHH', 2310.0), ('XIA', 2206.5), ('SHH', 2065.0), ('SHH', 2000.0), ('SHH', 1850.0), ('GZH', 1170.0), ('SHH', 1050.0), ('SHH', 915.07), ('GZH', 810.0), ('SHH', 746.73), ('XIA', 696.76), ('SHH', 366.97), ('SHH', 163.5), ('BJG', 135.09), ('XIA', 62.6)]<br>

1111111111
[ChatMessage(role=<MessageRole.USER: 'user'>, content="Given an input question, first create a syntactically correct sqlite query , then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a few relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:\n\nQuestion: Question here\nSQLQuery: SQL Query \nSQLResult: Result of the SQLQuery\nAnswer: Final answer here\n\nOnly use tables listed below.\nTable 'bop_test_cases' has columns: id (INTEGER), branch (VARCHAR(16)), payment_type (VARCHAR(16)), customer_tr

...

1111111111
[ChatMessage(role=<MessageRole.USER: 'user'>, content="Given an input question, first create a syntactically correct sqlite query , then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a few relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:\n\nQuestion: Question here\nSQLQuery: SQL Query \nSQLResult: Result of the SQLQuery\nAnswer: Final answer here\n\nOnly use tables listed below.\nTable 'bop_test_cases' has columns: id (INTEGER), branch (VARCHAR(16)), payment_type (VARCHAR(16)), customer_tr

**Node ID:** 601b4c8a-410d-4923-bfa6-7db9164d9357<br>**Similarity:** None<br>**Text:** [('SHH', 422104400.0), ('BJG', 90735372.47), ('BJG', 29300000.0), ('BJG', 11062394.84), ('BJG', 3400000.0), ('SHH', 1800546.42), ('SHH', 1400000.0), ('BJG', 811008.0), ('GZH', 802831.0), ('BJG', 609801.26), ('BJG', 562336.49), ('GZH', 537282.42), ('GZH', 490196.62), ('SHH', 391058.31), ('BJG', 316602.0), ('GZH', 300262.69), ('SHH', 264150.0), ('GZH', 258630.0), ('SHH', 258324.5), ('CQG', 216100.0), ('SHH', 171000.0), ('SHH', 111054.02), ('GZH', 99000.0), ('SHH', 97081.22), ('SHH', 77547.16), ('GZH', 70463.25), ('XIA', 68411.84), ('CQG', 67000.0), ('SHH', 64750.0), ('GZH', 53550.0), ('SHH', 53360.05), ('CQG', 53120.0), ('SHH', 49133.31), ('GZH', 43025.0), ('GZH', 41815.28), ('XIA', 39283.2), ('SHH', 38860.8), ('GZH', 36605.15), ('SHH', 33744.0), ('GZH', 30717.58), ('XIA', 28813.5), ('SHH', 28662.0), ('GZH', 24792.64), ('SHH', 21863.31)]<br>

1111111111
[ChatMessage(role=<MessageRole.USER: 'user'>, content="Given an input question, first create a syntactically correct sqlite query , then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a few relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:\n\nQuestion: Question here\nSQLQuery: SQL Query \nSQLResult: Result of the SQLQuery\nAnswer: Final answer here\n\nOnly use tables listed below.\nTable 'bop_test_cases' has columns: id (INTEGER), branch (VARCHAR(16)), payment_type (VARCHAR(16)), customer_tr

**Node ID:** 27484189-76e8-44e8-b7d3-5378da594c13<br>**Similarity:** None<br>**Text:** [('SHH', 422104400.0), ('BJG', 90735372.47), ('BJG', 29300000.0), ('BJG', 11062394.84), ('BJG', 3400000.0), ('SHH', 1800546.42), ('SHH', 1400000.0), ('BJG', 811008.0), ('GZH', 802831.0), ('BJG', 609801.26), ('BJG', 562336.49), ('GZH', 537282.42), ('GZH', 490196.62), ('SHH', 391058.31), ('BJG', 316602.0), ('GZH', 300262.69), ('SHH', 264150.0), ('GZH', 258630.0), ('SHH', 258324.5), ('CQG', 216100.0), ('SHH', 171000.0), ('SHH', 111054.02), ('GZH', 99000.0), ('SHH', 97081.22), ('SHH', 77547.16), ('GZH', 70463.25), ('XIA', 68411.84), ('CQG', 67000.0), ('SHH', 64750.0), ('GZH', 53550.0), ('SHH', 53360.05), ('CQG', 53120.0), ('SHH', 49133.31), ('GZH', 43025.0), ('GZH', 41815.28), ('XIA', 39283.2), ('SHH', 38860.8), ('GZH', 36605.15), ('SHH', 33744.0), ('GZH', 30717.58), ('XIA', 28813.5), ('SHH', 28662.0), ('GZH', 24792.64), ('SHH', 21863.31), ('SHH', 21685.0), ('CQG', 21585.0), ('SHH', 21294.9), ('TJN', 19570.0), ('SHH', 17022.78)]<br>

程序运行时间为： 55.73963952064514 秒


In [ ]:
from llama_index.core.retrievers import NLSQLRetriever

# default retrieval (return_raw=True)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["bop_test_cases"], return_raw=True,llm=llm
)

In [ ]:
results = nl_sql_retriever.retrieve(
    # "Return the top 5 branches (along with their amounts) with the smallest amount."
    "Return the branch and customer_transaction_postscript1 values for records where customer_transaction_code1 equals final_transaction_code1."
)


1111111111
[ChatMessage(role=<MessageRole.USER: 'user'>, content="Given an input question, first create a syntactically correct sqlite query , then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a few relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:\n\nQuestion: Question here\nSQLQuery: SQL Query \nSQLResult: Result of the SQLQuery\nAnswer: Final answer here\n\nOnly use tables listed below.\nTable 'bop_test_cases' has columns: id (INTEGER), branch (VARCHAR(16)), payment_type (VARCHAR(16)), customer_tr

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node

for n in results:
    display_source_node(n,source_length=1000)

**Node ID:** e46fb8ab-26fd-4ebb-884e-67b857cd91c7<br>**Similarity:** None<br>**Text:** [('BJG', '再保险支出财产险'), ('BJG', '偿还联属企业代垫不超过12个月外籍员工因公差旅费'), ('BJG', '计算机服务费'), ('BJG', '财务管理支持服务费 '), ('BJG', '外籍员工工资'), ('BJG', '公关服务费'), ('BJG', '非批量订购期刊'), ('BJG', '空运客运机票款'), ('BJG', '涉及我国出口的空中货运服务费'), ('BJG', '不涉及我国进出口的海洋货运服务费')]<br>

In [ ]:
# default retrieval (return_raw=False)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["bop_test_cases"], return_raw=False,llm=llm
)

In [ ]:
# NOTE: all the content is in the metadata
for n in results:
    display_source_node(n, show_source_metadata=True,source_length=1000)

**Node ID:** e46fb8ab-26fd-4ebb-884e-67b857cd91c7<br>**Similarity:** None<br>**Text:** [('BJG', '再保险支出财产险'), ('BJG', '偿还联属企业代垫不超过12个月外籍员工因公差旅费'), ('BJG', '计算机服务费'), ('BJG', '财务管理支持服务费 '), ('BJG', '外籍员工工资'), ('BJG', '公关服务费'), ('BJG', '非批量订购期刊'), ('BJG', '空运客运机票款'), ('BJG', '涉及我国出口的空中货运服务费'), ('BJG', '不涉及我国进出口的海洋货运服务费')]<br>**Metadata:** {}<br>

In [ ]:
from sqlalchemy import text
output = []
ground_truth = []
# 假设 engine 是你的数据库引擎对象
with engine.connect() as con:
    # 执行问题一的查询
    rows = con.execute(text("""
        SELECT
            ABS(sum_first_condition - sum_second_condition) AS amount_difference
        FROM
            (
                SELECT
                    SUM(final_transaction_amount1) AS sum_first_condition
                FROM
                    bop_test_cases
                WHERE
                    payment_type = '预付货款'
            ) first_sum,
            (
                SELECT
                    SUM(final_transaction_amount1) AS sum_second_condition
                FROM
                    bop_test_cases
                WHERE
                    branch = 'TJN'
            ) second_sum;
    """))
    for row in rows:
        output.append(row)
    # ground_truth.append(output)
    # output = []  

    # # 执行问题二的查询
    # rows = con.execute(text("""
    #     SELECT branch, customer_transaction_amount1
    #     FROM bop_test_cases
    #     ORDER BY customer_transaction_amount1 DESC
    #     LIMIT 5;
    # """))
    # for row in rows:
    #     output.append(row)
    # ground_truth.append(output)
    # output = []  
    # # 执行问题三的查询
    # rows = con.execute(text("""
    #     SELECT *
    #     FROM bop_test_cases
    #     WHERE branch = 'BJG' AND payment_type = '其他';
    # """))
    # for row in rows:
    #     output.append(row)
    # ground_truth.append(output)
    # output = []  
    print(output)

[(2294639.0,)]


In [ ]:
from sqlalchemy import text

output = []
ground_truth = []

# 假设 engine 是你的数据库引擎对象
with engine.connect() as con:
    # 遍历每个查询
    for query in sql_queries:
        rows = con.execute(text(query))
        for row in rows:
            output.append(row)
        ground_truth.append(output)
        output = []


In [ ]:
ground_truth = [str(x) for x in ground_truth]

In [ ]:
print(ground_truth)
print(outputs)

['[(123359031.24000001,)]', '[(439361402.58,)]', '[(2189777.73,)]', '[(124273240.87,)]', '[(439316222.58,)]', '[(437980972.58,)]', '[(124273240.87,)]', '[(125307659.87,)]', '[(2172749.76,)]', '[(136862489.65,)]', '[(424827305.13,)]', '[(424309137.82,)]', '[(424309137.82,)]', '[(427140599.69,)]', '[(122840863.93,)]', '[(313707731.71,)]', '[(2314074.54,)]', '[(360081.0,)]', '[(87903805.53,)]', '[(168.34000000000003,)]', '[(2168065.63,)]', '[(425741514.76,)]', '[(136497655.38,)]', '[(145228.93,)]', '[(235246.88,)]', '[(136717092.38,)]', '[(2060505.8599999999,)]', '[(90735267.4,)]', '[(11189995.50999999,)]', '[(439375543.15,)]', '[(439377441.58,)]', '[(70.56999999999971,)]', '[(2294639.63,)]', '[(2313462.9,)]', '[(440381142.77,)]', '[(134030859.44,)]', '[(439234828.58,)]', '[(439364127.21999997,)]', '[(125608490.87,)]', '[(293594.21,)]', '[(1085790.37,)]', '[(439234828.58,)]', '[(439361402.58,)]', '[(219437.0,)]', '[(439361402.58,)]', '[(237610.54,)]', '[(1102874.79,)]', '[(2168065.63,)]',

In [ ]:
cnt = 0
with open('output1.txt', 'w') as file:
    for i in range(len(outputs)):
        if ground_truth[i] == outputs[i]:
            cnt += 1
        else:
            file.write(f"Index: {i}\n")
            file.write(f"Output: {outputs[i]}\n")
            file.write(f"Ground Truth: {ground_truth[i]}\n")
            file.write("\n")  # 添加一个空行来分隔每个条目

# 打印匹配的计数
print(f"Number of matches: {cnt}")

Number of matches: 44


In [1]:
print( "/* Some SQL examples are provided based on similar problems: */\n/* Answer the following: How many employees do we have? */\nSELECT count(*) FROM Employee\n\n/* Answer the following: How many accounts do we have? */\nSELECT count(*) FROM Accounts\n\n/* Answer the following: How many regions do we have? */\nSELECT count(*) FROM region\n\n/* Answer the following: How many accounts do we have? */\nSELECT count(*) FROM Accounts\n\n/* Answer the following: How many activities do we have? */\nSELECT count(*) FROM Activity\n\n/* Answer the following: How many aircrafts do we have? */\nSELECT count(*) FROM Aircraft\n\n/* Answer the following: How many flights do we have? */\nSELECT count(*) FROM Flight\n\n/* Answer the following: How many customers do we have? */\nSELECT count(*) FROM Customers\n\n/* Answer the following: How many tracks do we have? */\nSELECT count(*) FROM track\n\n/* Given the following database schema: */\nCREATE TABLE \"stadium\" (\n\"Stadium_ID\" int,\n\"Location\" text,\n\"Name\" text,\n\"Capacity\" int,\n\"Highest\" int,\n\"Lowest\" int,\n\"Average\" int,\nPRIMARY KEY (\"Stadium_ID\")\n)\n\nCREATE TABLE \"singer\" (\n\"Singer_ID\" int,\n\"Name\" text,\n\"Country\" text,\n\"Song_Name\" text,\n\"Song_release_year\" text,\n\"Age\" int,\n\"Is_male\" bool,\nPRIMARY KEY (\"Singer_ID\")\n)\n\nCREATE TABLE \"concert\" (\n\"concert_ID\" int,\n\"concert_Name\" text,\n\"Theme\" text,\n\"Stadium_ID\" text,\n\"Year\" text,\nPRIMARY KEY (\"concert_ID\"),\nFOREIGN KEY (\"Stadium_ID\") REFERENCES \"stadium\"(\"Stadium_ID\")\n)\n\nCREATE TABLE \"singer_in_concert\" (\n\"concert_ID\" int,\n\"Singer_ID\" text,\nPRIMARY KEY (\"concert_ID\",\"Singer_ID\"),\nFOREIGN KEY (\"concert_ID\") REFERENCES \"concert\"(\"concert_ID\"),\nFOREIGN KEY (\"Singer_ID\") REFERENCES \"singer\"(\"Singer_ID\")\n)\n\n/* Answer the following: How many singers do we have? */\nSELECT ")

/* Some SQL examples are provided based on similar problems: */
/* Answer the following: How many employees do we have? */
SELECT count(*) FROM Employee

/* Answer the following: How many accounts do we have? */
SELECT count(*) FROM Accounts

/* Answer the following: How many regions do we have? */
SELECT count(*) FROM region

/* Answer the following: How many accounts do we have? */
SELECT count(*) FROM Accounts

/* Answer the following: How many activities do we have? */
SELECT count(*) FROM Activity

/* Answer the following: How many aircrafts do we have? */
SELECT count(*) FROM Aircraft

/* Answer the following: How many flights do we have? */
SELECT count(*) FROM Flight

/* Answer the following: How many customers do we have? */
SELECT count(*) FROM Customers

/* Answer the following: How many tracks do we have? */
SELECT count(*) FROM track

/* Given the following database schema: */
CREATE TABLE "stadium" (
"Stadium_ID" int,
"Location" text,
"Name" text,
"Capacity" int,
"Highest

In [ ]:
    {
        "db_id": "california_schools",
        "question": "What is the highest eligible free rate for K-12 students in the schools in Alameda County?",
        "evidence": "Eligible free rate for K-12 = `Free Meal Count (K-12)` / `Enrollment (K-12)`",
        "SQL": ""
    },


In [3]:
print({"db_id": "department_management", "query": "SELECT count(*) FROM head WHERE age  >  56", "query_toks": ["SELECT", "count", "(", "*", ")", "FROM", "head", "WHERE", "age", ">", "56"], "query_toks_no_value": ["select", "count", "(", "*", ")", "from", "head", "where", "age", ">", "value"], "question": "How many heads of the departments are older than 56 ?", "question_toks": ["How", "many", "heads", "of", "the", "departments", "are", "older", "than", "56", "?"], "sql": {"from": {"table_units": [["table_unit", 1]], "conds": []}, "select": [false, [[3, [0, [0, 0, false], null]]]], "where": [[false, 3, [0, [0, 10, false], null], 56.0, null]], "groupBy": [], "having": [], "orderBy": [], "limit": null, "intersect": null, "union": null, "except": null}}")

SyntaxError: invalid syntax (2298925954.py, line 1)